<a href="https://colab.research.google.com/github/Shamima5409/WSI-ViT-Heatmap/blob/main/WSI_project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RESULT This script now provides:✔ Full WSI tiling,✔ Background filtering,✔ PyTorch dataset + loader,✔ ViT-B/16 inference,✔ Heatmap on whole slide,✔ No collate / numpy errors,✔ Ready for real WSIs



In [3]:
#-------------------------------------------------------
# INSTALL DEPENDENCIES
#-------------------------------------------------------
!apt-get install -y openslide-tools >/dev/null
!pip install openslide-python torchvision tqdm >/dev/null

#-------------------------------------------------------
# DOWNLOAD A SMALL WSI
#-------------------------------------------------------
WSI_URL = "https://openslide.cs.cmu.edu/download/openslide-testdata/Aperio/CMU-1-Small-Region.svs"
WSI_NAME = "sample_wsi.svs"

!wget -q -O {WSI_NAME} {WSI_URL}
print("Downloaded:", WSI_NAME)

#-------------------------------------------------------
# IMPORTS
#-------------------------------------------------------
import openslide
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
import torch
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
import matplotlib.cm as cm

#-------------------------------------------------------
# CONFIG
#-------------------------------------------------------
PATCH_SIZE = 256
STEP = 256
BATCH_SIZE = 16
BG_WHITE_THRESH = 0.85
BG_RATIO_THRESH = 0.80
CLASS_OF_INTEREST = None   # If None → use top predicted class

#-------------------------------------------------------
# OPEN SLIDE
#-------------------------------------------------------
slide = openslide.OpenSlide(WSI_NAME)
print("WSI dimensions:", slide.dimensions)

#=======================================================
# (1) TILE COORDINATE GENERATOR
#=======================================================
def generate_coords(slide, size, step):
    W, H = slide.dimensions
    xs = list(range(0, W - size, step)) + [W - size]
    ys = list(range(0, H - size, step)) + [H - size]
    for y in ys:
        for x in xs:
            yield (x, y)

coords = list(generate_coords(slide, PATCH_SIZE, STEP))
print("Total tiles:", len(coords))

#=======================================================
# (2) BACKGROUND FILTER
#============================


Downloaded: sample_wsi.svs
WSI dimensions: (2220, 2967)
Total tiles: 108
